<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.3: 
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and databse maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands". 

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [87]:
import sqlite3
sqlite_db = 'test_db.sqlite' ### database name?
conn = sqlite3.connect(sqlite_db) ### connect object that represents the database
c = conn.cursor()    ### Cursor object to call its execute() method to perform SQL commands

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [88]:
c.execute('DROP TABLE houses;')   ### c.execute is to execute command. DROP TABLE drops (removes) the table

# Save (commit) the changes:
conn.commit()

In [89]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [90]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('''INSERT INTO houses (field1, sqft, bdrms, age, price) VALUES(?,?,?,?,?)''', last_sale)   ### does not work with None???

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key. 

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [91]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
# 

c.executemany('''INSERT INTO houses (field1, sqft, bdrms, age, price) VALUES(?,?,?,?,?)''', recent_sales)

conn.commit()

Select all rows from houses

In [92]:
# A
c.execute('SELECT * FROM houses')    ### MUST HAVE execute() to execute a command
conn.commit()
c.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more. 

In [93]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('housing-data.csv', dtype = 'i8', 
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [94]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [95]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [96]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [97]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [100]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

[]

In [102]:
r = c.execute('SELECT * FROM houses') ### why my output is blank []??? #cannot read from db statement
r.fetchall()

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000),
 (5, 2104, 3, 70, 399900),
 (6, 1600, 3, 28, 329900),
 (7, 2400, 3, 44, 369000),
 (8, 1416, 2, 49, 232000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (11, 1534, 3, 12, 314900),
 (12, 1427, 3, 57, 198999),
 (13, 1380, 3, 14, 212000),
 (14, 1494, 3, 15, 242500),
 (15, 1940, 4, 7, 239999),
 (16, 2000, 3, 27, 347000),
 (17, 1890, 3, 45, 329999),
 (18, 4478, 5, 49, 699900),
 (19, 1268, 3, 58, 259900),
 (20, 2300, 4, 77, 449900),
 (21, 1320, 2, 62, 299900),
 (22, 1236, 3, 78, 199900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (25, 1767, 3, 44, 252900),
 (26, 1888, 2, 79, 255000),
 (27, 1604, 3, 13, 242900),
 (28, 1962, 4, 53, 259900),
 (29, 3890, 3, 36, 573900),
 (30, 1100, 3, 60, 249900),
 (31, 1458, 3, 29, 464500),
 (32, 2526, 3, 13, 469000),
 (33, 2200, 3, 28, 475000),
 (34, 2637, 3, 25, 299900),
 (35, 1839, 2, 40, 349900),
 (36, 1000, 1, 5, 169900),
 (37

QUIZ: What would `DELETE FROM houses` do?
### delete data which contains 52 and 53?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [13]:
# ANSWER
c.execute('SELECT field1, sqft, bdrms, age, price FROM houses WHERE bdrms = 4');
conn.commit()
c.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. Run a query to calculate the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [14]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [15]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database. 

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [16]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [17]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

Run a query to get the average price of each house size from this table:

In [18]:
#ANSWER
avg_price = c.execute("SELECT AVG(price) AS avg_price FROM houses_pandas");
avg_price.fetchall()

[(340412.6595744681,)]

## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax 


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [123]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [124]:
# Reading CSV to Dataframe
orders = pd.read_csv('P12-ListOfOrders.csv', encoding = 'utf-8')
orders_break_down = pd.read_csv('P12-OrderBreakdown.csv', encoding = 'utf-8')

In [21]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [22]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [23]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]
orders

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
...,...,...,...,...
4112,ES-2014-2815584,2014-12-31,Erica Smith,Germany
4113,ES-2014-3458802,2014-12-31,John Grady,United Kingdom
4114,ES-2014-3638865,2014-12-31,Susan Vittorini,United Kingdom
4115,ES-2014-4785777,2014-12-31,Dennis Pardue,Germany


In [48]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]
orders_break_down

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies
...,...,...,...,...,...,...
8042,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture
8043,IT-2014-3715679,"BIC Pencil Sharpener, Fluorescent",0.5,30.0,2,Office Supplies
8044,IT-2014-3715679,"Avery Binder Covers, Recycled",0.5,23.0,4,Office Supplies
8045,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology


#### 3. Check DataTypes 

In [25]:
# ANSWER
orders.dtypes #dataframe

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [26]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [27]:
#LAWRENCE'S CODE
##first i created a db connection to connect and the cursor for these new tables
#sqlite_db2 = 'sqlite_db2.sqlite' #creates the db 'sqlite_db2'
#db_connection = sqlite3.connect('sqlite_db2')
#c2 = db_connection.cursor()

#then i created the tables and table schema
# Save these two dataframes as a table in sqlite
#create table orders first
#c2.execute('CREATE TABLE orders (field1 INTEGER PRIMARY KEY, order_id TEXT, date TEXT, customer_name TEXT, country TEXT);') ## this is the table schema, features/variables and data types after
#create 2nd table
#c2.execute('CREATE TABLE orderbreakdown (field1 INTEGER PRIMARY KEY, order_id TEXT, discount REAL, sales REAL, quantity INTEGER, category TEXT);') ## this is the table schema, features/variables and data types after
#conn.commit()

#the 2 commands below were to add the connect the df to the database
#df_orders.to_sql('orders', con=db_connection)
#df_orders_break_down.to_sql('ordersbreakdown', con=db_connection)

In [105]:
# Establishing Local DB connection
db_orderslist = 'orderslist.db' ###putting orders DF into this DB
db_connection = sqlite3.connect('orderslist.db');
db_cursor = db_connection.cursor()

# Save these two dataframes as a table in sqlite

# DROP TABLES cos it kept showing Error: table orders already exists
db_cursor.execute('DROP TABLE orders;')
db_cursor.execute('DROP TABLE orders_break_down;')
db_cursor.execute('DROP TABLE new_orders;')
db_cursor.execute('DROP TABLE new_orders_break_down;')
db_connection.commit()

# Create tables
db_cursor.execute('CREATE TABLE orders (field1 INTEGER PRIMARY KEY, order_id TEXT, order_date TEXT, customer_name TEXT, country TEXT);')
db_cursor.execute('CREATE TABLE orders_break_down (field1 INTEGER PRIMARY KEY, order_id TEXT, order_date TEXT, customer_name TEXT, country TEXT);')
db_connection.commit()

orders.to_sql('new_orders', con=db_connection) 
#orders here is the orders DataFrame in cell 25, we are putting it into 'new_orders' which is a table
orders_break_down.to_sql('new_orders_break_down', con=db_connection)
db_connection.commit()

In [107]:
print(new_orders)

None


In [130]:
# get name of table
query = ("""
        SELECT name FROM sqlite_master
        WHERE type IN ('table','view') AND name NOT LIKE 'sqlite_%'
        UNION ALL
        SELECT name FROM sqlite_temp_master
        WHERE type IN ('table','view')
        ORDER BY 1;
        """)
# query the table
db_cursor.execute(query).fetchall()

[('created_orders',),
 ('created_orders_break_down',),
 ('db_orders',),
 ('db_orders_break_down',),
 ('new_orders',),
 ('new_orders_break_down',),
 ('orders',),
 ('orders_2',),
 ('orders_3',),
 ('orders_break_down',),
 ('orders_break_down_2',),
 ('orders_break_down_3',)]

In [132]:
db_cursor.execute('DROP TABLE created_orders;')
db_cursor.execute('DROP TABLE created_orders_break_down;')
db_cursor.execute('DROP TABLE db_orders;')
db_cursor.execute('DROP TABLE db_orders_break_down;')
db_cursor.execute('DROP TABLE orders_2;')
db_cursor.execute('DROP TABLE orders_3;')
db_cursor.execute('DROP TABLE orders_break_down_2;')
db_cursor.execute('DROP TABLE orders_break_down_3;')
db_connection.commit()

OperationalError: no such table: created_orders

In [133]:
db_cursor.execute(query).fetchall()

[('new_orders',),
 ('new_orders_break_down',),
 ('orders',),
 ('orders_break_down',)]

In [121]:
#a = db_cursor.execute('SELECT * FROM new_orders')
#a.fetchall()


### how to check what is happening in this table?
### how to check if any tables/rows/columns are created?

print(type(orders))
print(type(new_orders))
print(orders)

<class 'pandas.core.frame.DataFrame'>
<class 'NoneType'>
             order_id  order_date      customer_name         country
0     IT-2011-3647632  2011-01-01       Eugene Moren          Sweden
1     ES-2011-4869686  2011-01-03  Dorothy Dickinson  United Kingdom
2     ES-2011-4939443  2011-01-04     Arthur Prichep          France
3     IT-2011-2942451  2011-01-04     Grant Thornton  United Kingdom
4     ES-2011-3848439  2011-01-05   Michael Granlund          France
...               ...         ...                ...             ...
4112  ES-2014-2815584  2014-12-31        Erica Smith         Germany
4113  ES-2014-3458802  2014-12-31         John Grady  United Kingdom
4114  ES-2014-3638865  2014-12-31    Susan Vittorini  United Kingdom
4115  ES-2014-4785777  2014-12-31      Dennis Pardue         Germany
4116  IT-2014-3715679  2014-12-31           Jim Kriz     Netherlands

[4117 rows x 4 columns]


In [113]:
b = db_cursor.execute('SELECT * FROM new_orders_break_down')
b.fetchall()

[(0,
  'IT-2011-3647632',
  'Enermax Note Cards, Premium',
  0.5,
  45.0,
  3,
  'Office Supplies'),
 (1,
  'ES-2011-4869686',
  'Dania Corner Shelving, Traditional',
  0.0,
  854.0,
  7,
  'Furniture'),
 (2,
  'ES-2011-4939443',
  'Binney & Smith Sketch Pad, Easy-Erase',
  0.0,
  140.0,
  3,
  'Office Supplies'),
 (3,
  'IT-2011-2942451',
  'Boston Markers, Easy-Erase',
  0.5,
  27.0,
  2,
  'Office Supplies'),
 (4,
  'IT-2011-2942451',
  'Eldon Folders, Single Width',
  0.5,
  17.0,
  2,
  'Office Supplies'),
 (5,
  'ES-2011-3848439',
  'Binney & Smith Pencil Sharpener, Water Color',
  0.0,
  90.0,
  3,
  'Office Supplies'),
 (6,
  'ES-2011-3848439',
  'Sanford Canvas, Fluorescent',
  0.0,
  207.0,
  4,
  'Office Supplies'),
 (7,
  'ES-2011-5433855',
  'Bush Floating Shelf Set, Pine',
  0.1,
  155.0,
  1,
  'Furniture'),
 (8,
  'ES-2011-5433855',
  'Accos Thumb Tacks, Assorted Sizes',
  0.0,
  33.0,
  3,
  'Office Supplies'),
 (9,
  'ES-2011-5433855',
  'Smead Lockers, Industrial',
 

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [35]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT * 
        FROM new_orders 
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,index,order_id,order_date,customer_name,country
0,0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [36]:
# Select Number of Orders for Each Customer

query2 = """SELECT *, COUNT(order_id) AS number_of_orders FROM new_orders GROUP BY customer_name"""

number_of_orders = sql.read_sql(query2, con=db_connection)
number_of_orders

,index,order_id,order_date,customer_name,country,number_of_orders
0,99,ES-2011-3528133,2011-03-21,Aaron Bergman,Italy,11
1,268,IT-2011-4320455,2011-06-17,Aaron Hawkins,France,7
2,100,ES-2011-5083176,2011-03-21,Aaron Smayling,Switzerland,12
3,1652,ES-2012-1205030,2012-12-19,Adam Bellavance,United Kingdom,4
4,248,IT-2011-4648014,2011-06-10,Adam Hart,Italy,8
...,...,...,...,...,...,...
787,1953,ES-2013-3285041,2013-04-27,Xylona Preis,Italy,2
788,558,ES-2011-4816650,2011-10-21,Yana Sorensen,Finland,5
789,94,ES-2011-5099069,2011-03-17,Yoseph Carroll,Germany,11
790,505,IT-2011-3183678,2011-09-23,Zuschuss Carroll,Germany,9


#### 6. Select Number of Customers for Each Country

In [42]:
# Select Number of Customers for Each Country

query3 = """SELECT country, COUNT(customer_name) AS number_of_cust FROM new_orders GROUP BY country ORDER BY 2 DESC"""

number_of_cust = sql.read_sql(query3, con=db_connection)
number_of_cust

,country,number_of_cust
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403
5,Netherlands,194
6,Austria,135
7,Sweden,100
8,Belgium,68
9,Ireland,50


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [49]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table
query4 = """SELECT order_id, discount, sales, quantity FROM new_orders_break_down"""
each_order = sql.read_sql(query4, con=db_connection)
each_order

,order_id,discount,sales,quantity
0,IT-2011-3647632,0.5,45.0,3
1,ES-2011-4869686,0.0,854.0,7
2,ES-2011-4939443,0.0,140.0,3
3,IT-2011-2942451,0.5,27.0,2
4,IT-2011-2942451,0.5,17.0,2
...,...,...,...,...
8042,ES-2014-3458802,0.0,245.0,2
8043,IT-2014-3715679,0.5,30.0,2
8044,IT-2014-3715679,0.5,23.0,4
8045,IT-2014-3715679,0.5,108.0,3


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [52]:
# Select discount, sales, quantity, total price for Each Order from orders_break_down Table
query6 = """SELECT order_id, ((sales*quantity)*(1-discount)) AS total_price FROM new_orders_break_down"""
total_price = sql.read_sql(query6, con=db_connection)
total_price

,order_id,total_price
0,IT-2011-3647632,67.5
1,ES-2011-4869686,5978.0
2,ES-2011-4939443,420.0
3,IT-2011-2942451,27.0
4,IT-2011-2942451,17.0
...,...,...
8042,ES-2014-3458802,490.0
8043,IT-2014-3715679,30.0
8044,IT-2014-3715679,46.0
8045,IT-2014-3715679,162.0


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [57]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100
query7 = """SELECT *, 
            ((sales*quantity)*(1-discount)) AS total_price 
            FROM new_orders_break_down 
            WHERE total_price > 100
            ORDER BY total_price"""
greater_than_100 = sql.read_sql(query7, con=db_connection)
greater_than_100

,index,order_id,product_name,discount,sales,quantity,category,total_price
0,2284,IT-2012-5640548,"Acme Trimmer, High Speed",0.5,67.0,3,Office Supplies,100.5
1,6436,IT-2014-1388520,"Stanley Sketch Pad, Easy-Erase",0.5,67.0,3,Office Supplies,100.5
2,1477,ES-2012-3480401,"Rogers Folders, Industrial",0.1,56.0,2,Office Supplies,100.8
3,1738,ES-2012-1678413,"Smead Folders, Blue",0.4,42.0,4,Office Supplies,100.8
4,6969,IT-2014-4304364,"Smead Folders, Single Width",0.4,42.0,4,Office Supplies,100.8
...,...,...,...,...,...,...,...,...
6245,3594,ES-2013-2860574,"KitchenAid Refrigerator, Black",0.0,5274.0,10,Office Supplies,52740.0
6246,5501,ES-2014-2637201,"Brother Fax Machine, Laser",0.0,4141.0,13,Technology,53833.0
6247,1834,ES-2012-2183106,"Canon Wireless Fax, High-Speed",0.1,4748.0,14,Technology,59824.8
6248,7590,ES-2014-3785216,"Hon Executive Leather Armchair, Adjustable",0.1,5729.0,14,Furniture,72185.4


#### 8. Select All Customer And The Product They Have Bought

In [61]:
# Select All Customer And The Product They Have Bought
query8 = """SELECT newo.customer_name AS cutomer_name, nobd.product_name AS product_name
            FROM new_orders_break_down nobd
            INNER JOIN new_orders newo
            ON newo.order_id = nobd.order_id"""
have_bought = sql.read_sql(query8, con=db_connection)
have_bought

,cutomer_name,product_name
0,Eugene Moren,"Enermax Note Cards, Premium"
1,Dorothy Dickinson,"Dania Corner Shelving, Traditional"
2,Arthur Prichep,"Binney & Smith Sketch Pad, Easy-Erase"
3,Grant Thornton,"Boston Markers, Easy-Erase"
4,Grant Thornton,"Eldon Folders, Single Width"
...,...,...
8042,John Grady,"Ikea Stackable Bookrack, Traditional"
8043,Jim Kriz,"BIC Pencil Sharpener, Fluorescent"
8044,Jim Kriz,"Avery Binder Covers, Recycled"
8045,Jim Kriz,"StarTech Phone, Red"


#### 9.A Select Number of 'Furniture' Orders For Each Country 

In [70]:
# Select Number of 'Furniture' Orders For Each Country
query9 = """
            SELECT newo.country AS country, COUNT(nobd.category) AS num_furniture
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id
            WHERE category = 'Furniture'
            GROUP BY country 
            ORDER BY num_furniture
            """
num_furniture = sql.read_sql(query9, con=db_connection)
num_furniture

,country,num_furniture
0,Norway,6
1,Denmark,9
2,Portugal,10
3,Switzerland,11
4,Ireland,12
5,Finland,16
6,Belgium,20
7,Sweden,36
8,Austria,40
9,Netherlands,76


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [71]:
# Select Number of 'Furniture' Orders For The Country Denmark
query10 = """
            SELECT newo.country AS country, COUNT(nobd.category) AS num_furniture
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id
            WHERE category = 'Furniture'
            AND country = 'Denmark'
            ORDER BY num_furniture
            """
num_furniture = sql.read_sql(query10, con=db_connection)
num_furniture

,country,num_furniture
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [73]:
# Select Total Sales With Discount and Without Discount for Each Country
query11 = """
            SELECT 
            newo.country AS country,
            ((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS totalsales_disc, 
            (nobd.sales*nobd.quantity) AS totalsales_no_disc
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id
            GROUP BY country
            """
sales = sql.read_sql(query11, con=db_connection)
sales

,country,totalsales_disc,totalsales_no_disc
0,Austria,55.0,55.0
1,Belgium,24.0,24.0
2,Denmark,25.5,51.0
3,Finland,780.0,780.0
4,France,420.0,420.0
5,Germany,34.0,34.0
6,Ireland,277.5,555.0
7,Italy,66.0,66.0
8,Netherlands,13.5,27.0
9,Norway,13.0,13.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [77]:
# Select Total Quantity, Total Sales for Each Country
query12 = """
            SELECT 
            newo.country AS country,
            ((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS totalsales_disc, 
            nobd.quantity AS total_qty
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id 
            GROUP BY country
            """
total_qty = sql.read_sql(query12, con=db_connection)
total_qty

,country,totalsales_disc,total_qty
0,Austria,55.0,1
1,Belgium,24.0,2
2,Denmark,25.5,3
3,Finland,780.0,4
4,France,420.0,3
5,Germany,34.0,2
6,Ireland,277.5,3
7,Italy,66.0,3
8,Netherlands,13.5,3
9,Norway,13.0,1


#### 11.B Select Top 3 Country Based on Sales

In [79]:
# Select Top 3 Country Based on Sales
query13 = """
            SELECT 
            newo.country AS country,
            ((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS totalsales_disc, 
            nobd.quantity AS total_qty
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id 
            GROUP BY country
            ORDER BY 2 DESC
            LIMIT 3
            """
top3_sales = sql.read_sql(query13, con=db_connection)
top3_sales

,country,totalsales_disc,total_qty
0,United Kingdom,5978.0,7
1,Portugal,855.0,3
2,Finland,780.0,4


#### 11.C Select Bottom 3 Country Based On Quantities

In [81]:
# Select Bottom 3 Country Based On Quantities
query13 = """
            SELECT 
            newo.country AS country,
            ((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS totalsales_disc, 
            nobd.quantity AS total_qty
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id 
            GROUP BY country
            ORDER BY 3
            LIMIT 3
            """
bottom3_qty = sql.read_sql(query13, con=db_connection)
bottom3_qty

,country,totalsales_disc,total_qty
0,Austria,55.0,1
1,Norway,13.0,1
2,Switzerland,19.0,1


#### 12. Select Average Sales By Categroy For The Country 'France'

In [83]:
# Select Average Sales By Categroy For The Country 'France'
query13 = """
            SELECT 
            newo.country AS country,
            AVG((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS avg_sales, 
            nobd.category AS category
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id 
            WHERE country = 'France'
            GROUP BY category
            ORDER BY 3
            """
avg_sales_FR = sql.read_sql(query13, con=db_connection)
avg_sales_FR

,country,avg_sales,category
0,France,2632.707860,Furniture
1,France,791.520390,Office Supplies
2,France,2508.548575,Technology


#### 13. Select Country, Category and Total Sales Where Average Total Sales is The Highest

In [86]:
# Select Country, Category and Total Sales Where Average Total Sales is The Highest
query14 = """
            SELECT 
            newo.country AS country,
            ((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS total_sales,
            AVG((nobd.sales*nobd.quantity)*(1-nobd.discount)) AS avg_sales, 
            nobd.category AS category
            FROM new_orders newo
            JOIN new_orders_break_down nobd
            ON newo.order_id = nobd.order_id 
            GROUP BY country
            ORDER BY 3 DESC
            LIMIT 1
            """
highest_avg_sales = sql.read_sql(query14, con=db_connection)
highest_avg_sales

,country,total_sales,avg_sales,category
0,Switzerland,19.0,1816.166667,Office Supplies


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



